<a href="https://colab.research.google.com/github/andrewbrr4/text-gen/blob/main/text_gen_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/drive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/datasets'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Mounted at /content/drive
/content/drive/MyDrive/datasets/david-bowie.txt
/content/drive/MyDrive/datasets/david-bowie-20-3/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-20-3/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-20-3/variables/variables.index
/content/drive/MyDrive/datasets/david-bowie-20-3/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-word/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-word/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-word/variables/variables.index
/content/drive/MyDrive/datasets/david-bowie-word/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-20-1/saved_model.pb
/content/drive/MyDrive/datasets/david-bowie-20-1/keras_metadata.pb
/content/drive/MyDrive/datasets/david-bowie-20-1/variables/variables.data-00000-of-00001
/content/drive/MyDrive/datasets/david-bowie-20-1/variables/variables.index
/content/drive/MyDrive/datasets/david-bow

In [ ]:
import tensorflow as tf
import keras
import re
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, SimpleRNN, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# Define the functions to create a model based off an artists lyrics and generate new lyrics from that model

In [ ]:
artist = 'david-bowie'
window = 20
step = 1

with open('/content/drive/MyDrive/datasets/david-bowie.txt') as f:
    text = f.read().lower()

text = re.sub('[^a-z \n]','',text)

text = re.sub(r'[\n]+', '\n',text)
text = text.replace('\n', ' \n ')

text = text.split(' ')

vocab = df = pd.DataFrame(text, columns = ['word'])
vocab = pd.DataFrame(df.value_counts()[:5000])
vocab = vocab.reset_index()
vocab = list(vocab['word'])

word_to_index = dict((w, i) for i, w in enumerate(vocab))
index_to_word = dict((i, w) for i, w in enumerate(vocab))

input_text = []
for item in text:
  if item in vocab:
    input_text.append(item)

inputs = []
next_words = []

for i in range(0,len(input_text)-window,step):
    inputs.append(input_text[i:i+window])
    next_words.append(input_text[i+window])
    
pairs = zip(inputs,next_words)
list(pairs)[:100]

X = np.zeros((len(inputs),window),dtype=np.float32)

for i in range(len(inputs)):
    for j in range(window):
        X[i,j] = word_to_index[inputs[i][j]]

y = np.zeros(len(next_words),dtype=np.float32)

for i in range(len(next_words)):
    y[i] = word_to_index[next_words[i]]

y = to_categorical(y)

In [ ]:
num_epochs = 50

model = Sequential()

model.add(Embedding(5000,64,input_length=window))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(5000,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
# model.fit(X,y,epochs=5,batch_size=20)
model.fit(X,y,epochs=num_epochs,batch_size=20)
model.save('/content/drive/MyDrive/datasets/'+artist+'-word')

Epoch 1/50
11711/11711 [==============================] - 1870s 159ms/step - loss: 5.5468 - accuracy: 0.1664
Epoch 2/50
11711/11711 [==============================] - 1894s 162ms/step - loss: 4.4867 - accuracy: 0.2541
Epoch 3/50
11711/11711 [==============================] - 1878s 160ms/step - loss: 3.6663 - accuracy: 0.3533
Epoch 4/50
11711/11711 [==============================] - 1872s 160ms/step - loss: 3.1422 - accuracy: 0.4242
Epoch 5/50
11711/11711 [==============================] - 1873s 160ms/step - loss: 2.7948 - accuracy: 0.4757
Epoch 6/50
11711/11711 [==============================] - 1897s 162ms/step - loss: 2.5552 - accuracy: 0.5115
Epoch 7/50
11711/11711 [==============================] - 1915s 164ms/step - loss: 2.3758 - accuracy: 0.5386
Epoch 8/50
11711/11711 [==============================] - 1938s 166ms/step - loss: 2.2408 - accuracy: 0.5596
Epoch 9/50
11711/11711 [==============================] - 1946s 166ms/step - loss: 2.1319 - accuracy: 0.5765
Epoch 10/50
11711/1

In [ ]:
bowie_bot = keras.models.load_model('/content/drive/MyDrive/datasets/david-bowie-word')

In [ ]:
def softmax(z):
    return np.exp(z)/sum(np.exp(z))

def temp_scale(conditional_probability, temperature=1.0):
    conditional_probability = np.asarray(conditional_probability).astype("float64")
    conditional_probability = np.log(conditional_probability) / temperature
    reweighted_conditional_probability = softmax(conditional_probability)
    probas = np.random.multinomial(1, reweighted_conditional_probability, 1)
    return np.argmax(probas)

def generate_line(model,context,temp,window):
    output = context
    line = context.split(' ')
    next_word = ''
    counter = 0
    while counter<50:
        X = [[word_to_index[line[i]] for i in range(0,len(line))]]
        X = pad_sequences(X,maxlen=window)
        y = model.predict(X)
        next_word = index_to_word[temp_scale(y[0,:],temp)]
        counter+=1
        line.append(next_word)
        output+=' '+next_word
    return output

temps = [.01, 0.2, 0.5, 1.0, 1.2, 1.5]

# Now lets have some fun emulating different artists!!

In [ ]:
output = '********** WINDOW = ' + str(window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'i want to',temp,window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics-word.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
i want to rest 
 because 
 youve got to pay you 
 when i met you when i met you 
 when i met you when i met you 
 when i met you when i met you 
 when i met you when i met you 
 when i met you

**** TEMP = 0.2 ****
i want to rest 
 because 
 youve got to pay you 
 when its good way to fall 
 just a couple of dreams 
 you get up and sleep 
 i got seven days to live my life 
 or seven ways to die 
 seven 
 seven 
 seven 


**** TEMP = 0.5 ****
i want to die 
 you know what you going to do 
 so just you are 
 oh what you will 
 but about a truth 
 love to the girl 
 dancing with the big boys 
 dancing with the big boys 
 well youve got your eyes and gimme my

**** TEMP = 1.0 ****
i want to go 
 and or it up the young american 
 late night right 
 there was a circus 
 and drugs afternoon 
 made me that the day must pulled you theres for his head my of 
 the empty man youve got his old boy 
 if of roll

**** TEMP = 1.2 ****
i

In [ ]:
output = '********** WINDOW = ' + str(window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'today',temp,window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics-word.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
today 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


**** TEMP = 0.2 ****
today 
 
 
 who can kick the home 
 but the 
 leaves 
 and the 
 and the next of the 
 
 and the ones 
 
 and the sky 
 and the glass 
 mr grownup 
 well the stars are out of the s 
 the

**** TEMP = 0.5 ****
today 
 
 i thought that you has 
 wish you well 
 doesnt matter doesnt matter 
 doesnt matter 
 doesnt matter doesnt matter 
 doesnt matter doesnt matter 
 i was not a star 
 i cant see his name 
 ill never see the water 
 yassassin aint

**** TEMP = 1.0 ****
today and this night oh then 
 hi cant 
 bing byebye small 
 pictures of lily 
 theyll be others blast the line 
 that i see eyes 
 cracked actor live   
 fame  makes me with the fire gang ohhh 
 dancing with the big boys

**** TEMP = 1.2 ****
today  
 youd break on my finger seen 
 doesnt matter about your head 
 when its detroit and

In [ ]:
output = '********** WINDOW = ' + str(window) + ' STEP = ' + str(step) + ' ***********\n'

for temp in temps:
    output += '**** TEMP = ' + str(temp) + ' ****\n'
    output += re.sub(
      r'[\n]+', '\n',generate_line(bowie_bot,'who am i',temp,window)
    )
    output += '\n\n'

print(output)
file = open("/content/drive/MyDrive/datasets/lyrics-word.txt", "a")  # append mode
file.write(output+'\n\n')
file.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


********** WINDOW = 20 STEP = 1 ***********
**** TEMP = 0.01 ****
who am i saw you 
 
 i hope you love 
 
 i was 
 
 i was 
 
 i am the young 
 i am with name 
 i am a dj i am what i play 
 cant turn around no cant turn around no cant turn around i

**** TEMP = 0.2 ****
who am i saw you 
 
 i took a trip 
 and he looks a 
 
 and i am the sky 
 i am the sky 
 i got a good ive got a photograph 
 i am the bridge 
 and i dig everything 
 i am a dj i

**** TEMP = 0.5 ****
who am i saw 
 watching your 
 memory of my life 
 and a london boys 
 thank you 
 this love is all we can 
 i know its all the same 
 and my love is better than any other 
 that i cant be the first 
 i cant

**** TEMP = 1.0 ****
who am i never had to 
 oh is 
 as find you 
 say rubber band 
 yeah and this used of her feeling 
 the tongues the fortune slept and the sunrise even he had my clean  in their songs song from the dream world or taxable over not 

**** TEMP = 1.2 ****
who am i do 
 dry 
 youre tomorrow 
 here will